In [1]:
import telegram
import xml.etree.ElementTree as et
import requests as req

In [2]:
# 텔레그램 인증키 등록
token = '2131981568:AAEeNzscHJkDMPHKd59X9tCvd-0UNckUx6c'

In [3]:
bot = telegram.Bot(token=token)

### 미세먼지 정보 알리미 봇
+ 미세먼지 : 죽음의 먼지, 잿빛 재앙, 은밀한 살인자 세계보건기구에 따르면, 2014년 기준, 전 세계에 약 700만명이 미세먼지로 인해 조기 사망

+ 1930년 벨기에의 뮤즈 벨리, 1948년 미국 펜실베니아 주의 도노라, 1952년 런던 스모그 등 대규모 재난을 겪으며 대기오염이 건강에 막대한 피해를 끼칠수 있다는 사실을 인지함

+ 먼지는 대기 중 떠다니거나 흩날려 내려오는 입자성 물질을 의미
+ 대기오염 물질에는 가스상 물질과 입자성 물질이 존재

+ 미세먼지, 초미세먼지는 입자상 물질을 크기별로 세분화한 것
    - 1990년대에는 입자의 지름이 50마이크로미터 이하를 총먼지로 분류
    - 2000년대에는 입자의 지름이 10마이크로미터 이하를 PM10으로 분류
    - 2010년대에는 입자의 지름이 2.5마이크로미터 이하를 PM2.5로 분류
        - 우리나라는 통상적으로 PM10을 미세먼지로
    - PM2.5를 초미세먼지로 번역해 옴

+ 한국환경공단이 제공하는 '대기오염정보 조회 서비스'를 통해
+ XML/JSON 형태로 받아온 데이터를 분석
+ 참고로 서울에는 총 25개의 구가 있음

In [4]:
# url1 == xml // url2 == json
url1 = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty?serviceKey=%2BpJJikYO7sLu5IRIGSVwrRhI2csJLgnXqbiIJsxf6n2g8HQ7IZwYWfBdGUhI5pclA093YhJpUyLroFhTYVfL8g%3D%3D&returnType=xml&numOfRows=100&pageNo=1&sidoName=%EC%84%9C%EC%9A%B8&ver=1.0'
url2 = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty?serviceKey=%2BpJJikYO7sLu5IRIGSVwrRhI2csJLgnXqbiIJsxf6n2g8HQ7IZwYWfBdGUhI5pclA093YhJpUyLroFhTYVfL8g%3D%3D&returnType=json&numOfRows=100&pageNo=1&sidoName=%EC%84%9C%EC%9A%B8&ver=1.0'

In [5]:
# xml 지정한 url로 요청해서 결과 받음
res = req.get(url1)
res.text[:500]

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response>\n  <header>\n    <resultCode>00</resultCode>\n    <resultMsg>NORMAL_CODE</resultMsg>\n  </header>\n  <body>\n    <items>\n      <item>\n        <so2Grade>1</so2Grade>\n        <coFlag/>\n        <khaiValue>47</khaiValue>\n        <so2Value>0.003</so2Value>\n        <coValue>0.3</coValue>\n        <pm25Flag/>\n        <pm10Flag/>\n        <o3Grade>1</o3Grade>\n        <pm10Value>32</pm10Value>\n        <khaiGrade>1</khaiGrade>\n        <pm25Value>8</pm25Value>\n   '

In [6]:
# 받은 결과를 파일로 저장
with open('/home/hadoop/telegram/air.xml','w',encoding='UTF-8') as f:
    f.writelines(res.text)

In [7]:
# xml 파일 읽어 XML 분석하고 DOM 트리를 구축함
tree = et.parse('/home/hadoop/telegram/air.xml')
root = tree.getroot()

In [8]:
# 측정일, 측정지역, 미세먼지, 초 미세먼지 값 출력
for data in root.findall('body/items/item'):
    date = data.find('dataTime').text
    stName = data.find('stationName').text
    pm10 = data.find('pm10Value').text
    pm25 = data.find('pm25Value').text
    print(date,stName,pm10,pm25)

2021-11-23 12:00 강남구 32 8
2021-11-23 12:00 서초구 47 9
2021-11-23 12:00 도산대로 42 8
2021-11-23 12:00 강남대로 39 9
2021-11-23 12:00 송파구 38 8
2021-11-23 12:00 강동구 36 9
2021-11-23 12:00 천호대로 43 10
2021-11-23 12:00 금천구 38 10
2021-11-23 12:00 시흥대로 - -
2021-11-23 12:00 강북구 46 11
2021-11-23 12:00 양천구 39 11
2021-11-23 12:00 노원구 34 6
2021-11-23 12:00 화랑로 31 8
2021-11-23 12:00 중구 33 9
2021-11-23 12:00 한강대로 37 8
2021-11-23 12:00 종로구 39 11
2021-11-23 12:00 청계천로 35 10
2021-11-23 12:00 종로 45 10
2021-11-23 12:00 용산구 42 13
2021-11-23 12:00 광진구 35 9
2021-11-23 12:00 성동구 39 8
2021-11-23 12:00 강변북로 44 9
2021-11-23 12:00 중랑구 36 8
2021-11-23 12:00 동대문구 35 9
2021-11-23 12:00 홍릉로 40 10
2021-11-23 12:00 성북구 38 9
2021-11-23 12:00 정릉로 38 11
2021-11-23 12:00 도봉구 38 7
2021-11-23 12:00 은평구 45 10
2021-11-23 12:00 서대문구 43 9
2021-11-23 12:00 마포구 39 11
2021-11-23 12:00 신촌로 33 8
2021-11-23 12:00 강서구 39 7
2021-11-23 12:00 공항대로 43 14
2021-11-23 12:00 구로구 38 5
2021-11-23 12:00 영등포구 34 8
2021-11-23 12:00 영등포로 34 9
2021-11-23 12:00

In [9]:
# 특정지역의 미세먼지 정보 출력
gu = '구로구'

for data in root.findall('body/items/item'):
    if (data.find('stationName').text == gu):
        date = data.find('dataTime').text
        stName = data.find('stationName').text
        pm10 = data.find('pm10Value').text
        pm25 = data.find('pm25Value').text
        print(date,stName,pm10,pm25)

2021-11-23 12:00 구로구 38 5


In [10]:
# 대기정보 수치에 따른 품질 평가
# 좋음😍 보통😄 나쁨🤨 최악😱
def rate_pm10(val):
    rate = '😱'
    if (val <= 30) : rate = '😍'
    elif (val <= 80) : rate = '😄'
    elif (val <= 150) : rate = '🤨'
    return rate

def rate_pm25(val):
    rate = '😱'
    if (val <= 15) : rate = '😍'
    elif (val <= 35) : rate = '😄'
    elif (val <= 75) : rate = '🤨'
    return rate

In [11]:
# 특정지역의 미세먼지 이모지 출력
for data in root.findall('body/items/item'):
    if (data.find('stationName').text == gu):
        date = data.find('dataTime').text
        stName = data.find('stationName').text
        pm10 = data.find('pm10Value').text
        pm25 = data.find('pm25Value').text
        p10 = rate_pm10(int(pm10))
        p25 = rate_pm25(int(pm25))
        print(date,stName,pm10, p10,pm25,p25)

2021-11-23 12:00 구로구 38 😄 5 😍


In [12]:
# jspn 지정한 url로 요청해서 결과 받음
res = req.get(url2)
res.text[:500]
# 받은 결과를 파일로 저장
with open('/home/hadoop/telegram/air.json','w',encoding='UTF-8') as f:
    f.writelines(res.text)

In [13]:
import json

In [14]:
# json 파일을 읽어 분석하고 객체(dict)로 저장
with open('/home/hadoop/telegram/air.json', encoding='UTF-8') as f:
    airdata = json.load(f)

In [15]:
# 측정일, 측정지역, 미세먼지, 초 미세먼지 값 출력
for item in airdata['response']['body']['items']:
    date = item['dataTime']
    stName = item['stationName']
    pm10 = item['pm10Value']
    pm25 = item['pm25Value']
    print(date,stName,pm10,pm25)

2021-11-23 12:00 강남구 32 8
2021-11-23 12:00 서초구 47 9
2021-11-23 12:00 도산대로 42 8
2021-11-23 12:00 강남대로 39 9
2021-11-23 12:00 송파구 38 8
2021-11-23 12:00 강동구 36 9
2021-11-23 12:00 천호대로 43 10
2021-11-23 12:00 금천구 38 10
2021-11-23 12:00 시흥대로 - -
2021-11-23 12:00 강북구 46 11
2021-11-23 12:00 양천구 39 11
2021-11-23 12:00 노원구 34 6
2021-11-23 12:00 화랑로 31 8
2021-11-23 12:00 중구 33 9
2021-11-23 12:00 한강대로 37 8
2021-11-23 12:00 종로구 39 11
2021-11-23 12:00 청계천로 35 10
2021-11-23 12:00 종로 45 10
2021-11-23 12:00 용산구 42 13
2021-11-23 12:00 광진구 35 9
2021-11-23 12:00 성동구 39 8
2021-11-23 12:00 강변북로 44 9
2021-11-23 12:00 중랑구 36 8
2021-11-23 12:00 동대문구 35 9
2021-11-23 12:00 홍릉로 40 10
2021-11-23 12:00 성북구 38 9
2021-11-23 12:00 정릉로 38 11
2021-11-23 12:00 도봉구 38 7
2021-11-23 12:00 은평구 45 10
2021-11-23 12:00 서대문구 43 9
2021-11-23 12:00 마포구 39 11
2021-11-23 12:00 신촌로 33 8
2021-11-23 12:00 강서구 39 7
2021-11-23 12:00 공항대로 43 14
2021-11-23 12:00 구로구 38 5
2021-11-23 12:00 영등포구 34 8
2021-11-23 12:00 영등포로 34 9
2021-11-23 12:00

In [16]:
# 특정지역의 미세먼지 이모지 출력
for item in airdata['response']['body']['items']:
    if (item['stationName'] == gu):
        date = item['dataTime']
        stName = item['stationName']
        pm10 = item['pm10Value']
        pm25 = item['pm25Value']
        p10 = rate_pm10(int(pm10))
        p25 = rate_pm25(int(pm25))
        print(date,stName,pm10, p10,pm25,p25)

2021-11-23 12:00 구로구 38 😄 5 😍


In [17]:
# 텔레그램으로 미세먼지 정보 서비스 하기
def reload_air(): # data.go.kr 에서 대기정보 받아오기
    res = req.get(url2)
    with open('/home/hadoop/telegram/air.json', encoding='UTF-8') as f:
        airdata = json.load(f)

def fetch_air(gu): # 특정지역의 미세먼지 정보/이모지 받아오기
    with open('/home/hadoop/telegram/air.json', encoding='UTF-8') as f:
        airdata = json.load(f)
    
    airinfo = ''
    for item in airdata['response']['body']['items']:
        if (item['stationName'] == gu):
            date = item['dataTime']
            pm10 = item['pm10Value']
            pm25 = item['pm25Value']
            p10 = rate_pm10(int(pm10))
            p25 = rate_pm25(int(pm25))
            airinfo = f'{date} {pm10} {p10} {pm25} {p25}'
    return airinfo

In [18]:
from telegram import Update
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters
from telegram.ext import Updater, CallbackContext

In [21]:
# 특정 대화상대에게 메시지 보내기 테스트
# sendMessage(대상, 메시지)
bot.sendMessage(chat_id='2110692703', text='hello, World!')

In [22]:
# 대기정보 출력
def air_message(update, context):
    airinfo = fetch_air(update.message.text)
    context.bot.send_message(update.effective_chat.id, airinfo)

In [23]:
# 대기정보 갱신
def reload_message(update, context):
    reload_air()
    context.bot.send_message(update.effective_chat.id, '대기정보 갱신됨!!')

In [24]:
# 텔레그램 챗봇 서버 초기화
updater = Updater(token)

# 핸들러 생성
air_handler = MessageHandler(Filters.text, air_message)
reload_handler = CommandHandler('reload', reload_message)

# 핸들러 등록
updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(air_handler)

In [27]:
# 텔레그램 챗봇 서버 시작
print('텔레그램 대기정보 RPA 서버 실행중...')
updater.start_polling(timeout=3, clean=True)
updater.idle()
# 텔레그램 구동 -> /reload -> 구이름

텔레그램 대기정보 RPA 서버 실행중...


/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
